![<caption>](UFPR.png)

Disciplina: Linguagem de Programação Aplicada - Linguagem R 

Professor: Dr. Razer A N R Montaño

O que deve ser entregue:

- Um arquivo compactado com os documentos e arquivos

- A lista de comandos R que foi executada, com suas respectivas saídas

- Um texto com o resultado e justificativa do porquê 

- Outros arquivos pedidos (ex. modelo gerado)

### PESQUISA COM DADOS DE SATÉLITE (Satellite)

O banco de dados consiste nos valores multi-espectrais de pixels em vizinhanças 3x3 em uma imagem de satélite, e na classificação associada ao pixel central em cada vizinhança.
O objetivo é prever esta classificação, dados os valores multi-espectrais. 

Um quadro de imagens do Satélite Landsat com MSS (Multispectral Scanner System) consiste em quatro imagens digitais da mesma cena em diferentes bandas espectrais. Duas delas estão na região visível (correspondendo aproximadamente às regiões verde e
vermelha do espectro visível) e duas no infravermelho (próximo). Cada pixel é uma palavra binária de 8 bits, com 0 correspondendo a preto e 255 a branco. A resolução espacial de um
pixel é de cerca de 80m x 80m. Cada imagem contém 2340 x 3380 desses pixels. O banco de dados é uma subárea (minúscula) de uma cena, consistindo de 82 x 100 pixels. Cada
linha de dados corresponde a uma vizinhança quadrada de pixels 3x3 completamente contida dentro da subárea 82x100. Cada linha contém os valores de pixel nas quatro
bandas espectrais (convertidas em ASCII) de cada um dos 9 pixels na vizinhança de 3x3 e um número indicando o rótulo de classificação do pixel central.

As classes são: solo vermelho, colheita de algodão, solo cinza, solo cinza úmido, restolho de vegetação, solo cinza muito úmido.

Os dados estão em ordem aleatória e certas linhas de dados foram removidas, portanto você não pode reconstruir a imagem original desse conjunto de dados. Em cada linha de dados, os quatro valores espectrais para o pixel superior esquerdo são dados primeiro, seguidos pelos quatro valores espectrais para o pixel superior central e, em seguida, para o pixel superior direito, e assim por diante, com os pixels lidos em sequência, da esquerda para a direita e de cima para baixo. Assim, os quatro valores espectrais para o pixel central são dados pelos atributos 17, 18, 19 e 20. Se você quiser, pode usar apenas esses quatro
atributos, ignorando os outros. Isso evita o problema que surge quando uma vizinhança 3x3 atravessa um limite.

O banco de dados se encontra no pacote mlbench e é completo (não possui dados faltantes).

Tarefas:
  
1 - Treine modelos RandomForest, SVM e RNA para predição destes dados. 

2 - Escolha o melhor modelo com base em suas matrizes de confusão. 

3 - Treine o modelo final com todos os dados e faça a predição na base completa. 

4 - Analise o resultado.

5 - Salve este modelo final 

In [1]:
setwd("C:/FCD/UFPR")
getwd()

[1] "C:/FCD/UFPR"

In [2]:
# DESATIVAR MENSAGENS DE WARNINGS (warn = -1) E ATIVAR (warn = 0)
options(warn = -1)

In [3]:
suppressPackageStartupMessages({library(mlbench)
library(dplyr)
library(rpart)
library(e1071)
library(caret)
library(caTools)
library(neuralnet)
library(kernlab)
library(data.table)})

In [4]:
set.seed(1971)

In [5]:
# DADOS
data(Satellite)
head(Satellite)

x.1,x.2,x.3,x.4,x.5,x.6,x.7,x.8,x.9,x.10,...,x.28,x.29,x.30,x.31,x.32,x.33,x.34,x.35,x.36,classes
92,115,120,94,84,102,106,79,84,102,...,104,88,121,128,100,84,107,113,87,grey soil
84,102,106,79,84,102,102,83,80,102,...,100,84,107,113,87,84,99,104,79,grey soil
84,102,102,83,80,102,102,79,84,94,...,87,84,99,104,79,84,99,104,79,grey soil
80,102,102,79,84,94,102,79,80,94,...,79,84,99,104,79,84,103,104,79,grey soil
84,94,102,79,80,94,98,76,80,102,...,79,84,103,104,79,79,107,109,87,grey soil
80,94,98,76,80,102,102,79,76,102,...,79,79,107,109,87,79,107,109,87,grey soil


In [6]:
str(Satellite)

'data.frame':	6435 obs. of  37 variables:
 $ x.1    : num  92 84 84 80 84 80 76 76 76 76 ...
 $ x.2    : num  115 102 102 102 94 94 102 102 89 94 ...
 $ x.3    : num  120 106 102 102 102 98 106 106 98 98 ...
 $ x.4    : num  94 79 83 79 79 76 83 87 76 76 ...
 $ x.5    : num  84 84 80 84 80 80 76 80 76 76 ...
 $ x.6    : num  102 102 102 94 94 102 102 98 94 98 ...
 $ x.7    : num  106 102 102 102 98 102 106 106 98 102 ...
 $ x.8    : num  79 83 79 79 76 79 87 79 76 72 ...
 $ x.9    : num  84 80 84 80 80 76 80 76 76 76 ...
 $ x.10   : num  102 102 94 94 102 102 98 94 98 94 ...
 $ x.11   : num  102 102 102 98 102 102 106 102 102 90 ...
 $ x.12   : num  83 79 79 76 79 79 79 76 72 76 ...
 $ x.13   : num  101 92 84 84 84 76 80 80 80 76 ...
 $ x.14   : num  126 112 103 99 99 99 107 112 95 91 ...
 $ x.15   : num  133 118 104 104 104 104 118 118 104 104 ...
 $ x.16   : num  103 85 81 78 81 81 88 88 74 74 ...
 $ x.17   : num  92 84 84 84 76 76 80 80 76 76 ...
 $ x.18   : num  112 103 99 99 99 99

In [7]:
# TRANSFORMANDO A COLUNA classes EM FACTOR COM NÚMEROS.
df_satellite <- Satellite %>%
  mutate(classes2 = factor(as.numeric(Satellite$classes)))

df_satellite <- df_satellite[,c(-37)]

In [8]:
head(df_satellite)

x.1,x.2,x.3,x.4,x.5,x.6,x.7,x.8,x.9,x.10,...,x.28,x.29,x.30,x.31,x.32,x.33,x.34,x.35,x.36,classes2
92,115,120,94,84,102,106,79,84,102,...,104,88,121,128,100,84,107,113,87,3
84,102,106,79,84,102,102,83,80,102,...,100,84,107,113,87,84,99,104,79,3
84,102,102,83,80,102,102,79,84,94,...,87,84,99,104,79,84,99,104,79,3
80,102,102,79,84,94,102,79,80,94,...,79,84,99,104,79,84,103,104,79,3
84,94,102,79,80,94,98,76,80,102,...,79,84,103,104,79,79,107,109,87,3
80,94,98,76,80,102,102,79,76,102,...,79,79,107,109,87,79,107,109,87,3


In [9]:
levels(df_satellite$classes2)

[1] "1" "2" "3" "4" "5" "6"

As classes são: solo vermelho (1), colheita de algodão (2), solo cinza (3), solo cinza úmido (4), restolho de vegetação (5), solo cinza muito úmido (6).

In [10]:
# DADOS TREINO E TESTE 80% 20%
treino_teste <- sample.split(df_satellite$classes2, SplitRatio = 0.80)
dados_treino <- subset(df_satellite, treino_teste == TRUE)
dados_teste <- subset(df_satellite, treino_teste == FALSE)

In [11]:
cat("Dados de treino:", nrow(dados_treino),"linhas","\nDados de teste:", nrow(dados_teste),"linhas")

Dados de treino: 5147 linhas 
Dados de teste: 1288 linhas

### 1 - TREINE MODELOS RANDOM FOREST, SVM E RNA PARA PREDIÇÃO DESTES DADOS.
#### 1.1 - MODELO RANDOM FOREST

In [12]:
# FÓRMULA
formula <- classes2 ~.

In [13]:
# TREINAMENTO DO MODELO
modelo_rf <- rpart(formula = formula, data = dados_treino, control = rpart.control(cp = 0.0005))

In [14]:
saveRDS(modelo_rf, "modelo_rf.rds")

In [15]:
modelo_rf <- readRDS("modelo_rf.rds")

In [16]:
# PREDIÇÃO DO MODELO
previsao_rf <- predict(modelo_rf, dados_teste, type = 'class')

In [17]:
# MATRIZ DE CONFUSÃO
matriz_confusao1 <- confusionMatrix(previsao_rf, dados_teste$classes2)
matriz_confusao1

Confusion Matrix and Statistics

          Reference
Prediction   1   2   3   4   5   6
         1 292   4   1   2  10   2
         2   2 132   0   0   2   3
         3   6   0 256  24   0  10
         4   0   1  12  67   3  21
         5   6   4   0   2 112   4
         6   1   0   3  30  14 262

Overall Statistics
                                          
               Accuracy : 0.8703          
                 95% CI : (0.8508, 0.8882)
    No Information Rate : 0.2384          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.8393          
                                          
 Mcnemar's Test P-Value : NA              

Statistics by Class:

                     Class: 1 Class: 2 Class: 3 Class: 4 Class: 5 Class: 6
Sensitivity            0.9511   0.9362   0.9412  0.53600  0.79433   0.8675
Specificity            0.9806   0.9939   0.9606  0.96819  0.98605   0.9513
Pos Pred Value         0.9389   0.9496   0.8649

#### 1.2 - MODELO SUPPORT VECTOR MACHINES - SVM 

In [18]:
# TREINAMENTO DO MODELO
modelo_svm <- svm(formula = formula, data = dados_treino, type = 'C-classification', 
                  kernel = 'radial')

In [19]:
saveRDS(modelo_svm, "modelo_svm.rds")

In [20]:
modelo_svm <- readRDS("modelo_svm.rds")

In [21]:
summary(modelo_svm)


Call:
svm(formula = formula, data = dados_treino, type = "C-classification", 
    kernel = "radial")


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  radial 
       cost:  1 

Number of Support Vectors:  1657

 ( 265 467 252 451 91 131 )


Number of Classes:  6 

Levels: 
 1 2 3 4 5 6




In [22]:
# PREDIÇÃO DO MODELO
previsao_svm <- predict(modelo_svm, dados_teste)

In [23]:
# MATRIZ DE CONFUSÃO
matriz_confusao2 <- confusionMatrix(previsao_svm, dados_teste$classes2)
matriz_confusao2

Confusion Matrix and Statistics

          Reference
Prediction   1   2   3   4   5   6
         1 300   0   2   1   4   0
         2   0 139   1   1   1   0
         3   5   0 256  18   1   6
         4   0   0  11  83   1  33
         5   2   2   0   2 123   5
         6   0   0   2  20  11 258

Overall Statistics
                                          
               Accuracy : 0.8998          
                 95% CI : (0.8821, 0.9157)
    No Information Rate : 0.2384          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.8764          
                                          
 Mcnemar's Test P-Value : NA              

Statistics by Class:

                     Class: 1 Class: 2 Class: 3 Class: 4 Class: 5 Class: 6
Sensitivity            0.9772   0.9858   0.9412  0.66400   0.8723   0.8543
Specificity            0.9929   0.9974   0.9705  0.96131   0.9904   0.9665
Pos Pred Value         0.9772   0.9789   0.8951

#### 1.3 - MODELO REDES NEURAIS ARTIFICIAIS - RNA

In [24]:
# TREINAMENTO DO MODELO
modelo_rna <- train(formula, dados_treino, method = "nnet", trace = FALSE)
modelo_rna

Neural Network 

5147 samples
  36 predictor
   6 classes: '1', '2', '3', '4', '5', '6' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 5147, 5147, 5147, 5147, 5147, 5147, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa     
  1     0e+00  0.2724924  0.05337293
  1     1e-04  0.2331361  0.00000000
  1     1e-01  0.3771697  0.19527957
  3     0e+00  0.3286605  0.13110565
  3     1e-04  0.3022562  0.09489613
  3     1e-01  0.5021745  0.36537354
  5     0e+00  0.3562555  0.16529654
  5     1e-04  0.3596942  0.17421837
  5     1e-01  0.5649937  0.44843302

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 5 and decay = 0.1.

In [25]:
saveRDS(modelo_rna, "modelo_rna.rds")

In [26]:
modelo_rna <- readRDS("modelo_rna.rds")

In [27]:
# PREVISÃO DO MODELO
previsao_rna <- predict(modelo_rna, dados_teste)

In [28]:
# MATRIZ DE CONFUSÃO
matriz_confusao3 <- confusionMatrix(previsao_rna, dados_teste$classes2)
matriz_confusao3

Confusion Matrix and Statistics

          Reference
Prediction   1   2   3   4   5   6
         1   0   0   0   0   0   0
         2   0 120   0   0   0   0
         3   0   0   0   0   0   0
         4   0   0   0   0   0   0
         5   1  21   1   3 112  11
         6 306   0 271 122  29 291

Overall Statistics
                                          
               Accuracy : 0.4061          
                 95% CI : (0.3791, 0.4335)
    No Information Rate : 0.2384          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.2497          
                                          
 Mcnemar's Test P-Value : NA              

Statistics by Class:

                     Class: 1 Class: 2 Class: 3 Class: 4 Class: 5 Class: 6
Sensitivity            0.0000  0.85106   0.0000  0.00000  0.79433   0.9636
Specificity            1.0000  1.00000   1.0000  1.00000  0.96774   0.2617
Pos Pred Value            NaN  1.00000      NaN

### 2 - ESCOLHA O MELHOR MODELO COM BASE EM SUAS MATRIZES DE CONFUSÃO.

Com base na matriz de confusão, o melhor modelo foi o SVM com uma acurácia de 0.8998

### 3 - TREINE O MODELO FINAL COM TODOS OS DADOS E FAÇA A PREDIÇÃO NA BASE COMPLETA.

In [29]:
# TREINAMENTO DO MODELO
modelo_svm_final <- svm(formula = formula, data = df_satellite, type = 'C-classification', 
                        kernel = 'radial')

In [30]:
saveRDS(modelo_svm_final, "modelo_svm_final.rds")

In [31]:
modelo_svm_final <- readRDS("modelo_svm_final.rds")

In [32]:
# PREDIÇÃO DO MODELO
previsao_svm_final <- predict(modelo_svm_final, df_satellite)

In [33]:
# MATRIZ DE CONFUSÃO
matriz_confusao2_final <- confusionMatrix(previsao_svm_final, df_satellite$classes2)
matriz_confusao2_final

Confusion Matrix and Statistics

          Reference
Prediction    1    2    3    4    5    6
         1 1517    0    8    4   20    0
         2    2  691    2    2    6    1
         3   11    0 1316  127    2   31
         4    0    3   28  389    4  126
         5    3    6    1    7  632   30
         6    0    3    3   97   43 1320

Overall Statistics
                                          
               Accuracy : 0.9114          
                 95% CI : (0.9042, 0.9183)
    No Information Rate : 0.2382          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.8905          
                                          
 Mcnemar's Test P-Value : NA              

Statistics by Class:

                     Class: 1 Class: 2 Class: 3 Class: 4 Class: 5 Class: 6
Sensitivity            0.9896   0.9829   0.9691  0.62141  0.89392   0.8753
Specificity            0.9935   0.9977   0.9663  0.97228  0.99179   0.9704
Pos P

### 4 - ANÁLISE O RESULTADO.

A acurácia do modelo SVM final foi maior que a do primeiro SVM, mas acredito que o modelo final SVM não seja confiável, pois foram apresentados todos os dados ao algoritmo treinamento, conforme descrição do exercício. Os dados de teste não foram apresentados. Pode ser que o modelo final não se comporte bem com novos dados. 

### 5 - SALVE ESTE MODELO FINAL.

In [34]:
saveRDS(modelo_svm_final, "modelo_svm_final.rds")

### ESTIMATIVA DE VOLUMES DE ÁRVORES.

Modelos de aprendizado de máquina são bastante usados na área da engenharia florestal (mensuração florestal) para, por exemplo, estimar o volume de madeira de árvores sem ser necessário abatê-las.

O processo é feito pela coleta de dados (dados observados) através do abate de algumas árvores, onde sua altura, diâmetro na altura do peito (dap), etc, são medidos de forma exata. Com estes dados, treina-se um modelo de AM que pode estimar o volume de outras árvores da população.

Os modelos, chamados alométricos, são usados na área há muitos anos e são baseados em regressão (linear ou não) para encontrar uma equação que descreve os dados. Por exemplo, o modelo de Spurr é dado por:

                                              Volume = b0 + b1 * dap² * Ht

Onde dap é o diâmetro na altura do peito (1,3 metros), Ht é a altura total. Tem-se vários modelos alométricos, cada um com uma determinada característica, parâmetros, etc. Um modelo de regressão envolve aplicar os dados observados e encontrar b0 e b1 no modelo apresentado, gerando assim uma equação que pode ser usada para prever o volume de outras árvores.

Dado o arquivo Volumes.csv, que contém os dados de observação, escolha um modelo de aprendizado de máquina com a melhor estimativa, a partir da estatística de correlação.


Tarefas:

1 - Carregar o arquivo Volumes.csv (http://www.razer.net.br/datasets/Volumes.csv).

2 - Eliminar a coluna NR, que só apresenta um número sequencial.

3 - Criar partição de dados: treinamento 80%, teste 20%.

4 - Usando o pacote "caret", treinar os modelos: Random Forest (rf), SVM (svmRadial), Redes Neurais (neuralnet) e o modelo alométrico de SPURR.

5 - O modelo alométrico é dado por: Volume = b0 + b1 * dap² * Ht

                  alom <- nls(VOL ~ b0 + b1*DAP*DAP*HT, dados, start=list(b0 = 0.5,b1 = 0.5))

6 - Efetue as predições nos dados de teste

7 - Crie funções e calcule as seguintes métricas entre a predição e os dados observados.

Coeficiente de determinação (R²): esta métrica indica erro, portanto quanto mais perto de 0 melhor é o modelo.

Erro padrão da estimativa (Syx): esta métrica indica porcentagem de erro, portanto quanto mais perto de 0 melhor é o
modelo.

Erro padrão da estimativa percentual (Syx%): esta métrica indica porcentagem de erro, portanto quanto mais perto de 0 melhor é o modelo.

8 - Escolha o melhor modelo.

#### 1 - CARREGAR O ARQUIVO Volumes.csv (http://www.razer.net.br/datasets/Volumes.csv).

In [35]:
df_volume <- as.data.frame(fread("http://www.razer.net.br/datasets/Volumes.csv", 
                                 stringsAsFactors = FALSE, dec=","))
head(df_volume, 10)

NR,DAP,HT,HP,VOL
1,34.0,27.00,1.80,0.8971441
2,41.5,27.95,2.75,1.6204441
3,29.6,26.35,1.15,0.8008181
4,34.3,27.15,1.95,1.0791682
5,34.5,26.20,1.00,0.9801112
6,29.9,27.10,1.90,0.9067022
7,28.4,25.40,2.30,0.7731585
8,29.5,25.50,2.40,0.8715001
9,36.3,27.00,1.80,1.3132469
10,36.3,26.70,1.50,1.3868891


In [36]:
str(df_volume)

'data.frame':	100 obs. of  5 variables:
 $ NR : int  1 2 3 4 5 6 7 8 9 10 ...
 $ DAP: num  34 41.5 29.6 34.3 34.5 29.9 28.4 29.5 36.3 36.3 ...
 $ HT : num  27 27.9 26.4 27.1 26.2 ...
 $ HP : num  1.8 2.75 1.15 1.95 1 1.9 2.3 2.4 1.8 1.5 ...
 $ VOL: num  0.897 1.62 0.801 1.079 0.98 ...


#### 2 - ELIMINAR A COLUNA NR, QUE SÓ APRESENTA UM NÚMERO SEQUÊNCIA. 

In [37]:
df_volume$NR <- NULL

In [38]:
head(df_volume, 10)

DAP,HT,HP,VOL
34.0,27.00,1.80,0.8971441
41.5,27.95,2.75,1.6204441
29.6,26.35,1.15,0.8008181
34.3,27.15,1.95,1.0791682
34.5,26.20,1.00,0.9801112
29.9,27.10,1.90,0.9067022
28.4,25.40,2.30,0.7731585
29.5,25.50,2.40,0.8715001
36.3,27.00,1.80,1.3132469
36.3,26.70,1.50,1.3868891


#### 3 - CRIAR PARTIÇÃO DE DADOS: TREINAMENTO 80% TESTE 20%.

In [39]:
treino_teste2 <- sample.split(df_volume$VOL, SplitRatio = 0.8)
dados_treino2 <- subset(df_volume, treino_teste2 == TRUE)
dados_teste2 <- subset(df_volume, treino_teste2 == FALSE)

In [40]:
write.csv(dados_treino2, "dados_treino2.cvs")
write.csv(dados_teste2, "dados_teste2.cvs")

In [41]:
dados_treino2 <- read.csv("dados_treino2.cvs")
dados_teste2 <- read.csv("dados_teste2.cvs")

In [42]:
nrow(dados_treino2)
nrow(dados_teste2)

[1] 80

[1] 20

#### 4 - USANDO O PACOTE "caret", TREINAR OS MODELOS: RANDOM FOREST (rf), SVM (svmRadial), REDES NEURAIS (neuralnet) E O MODELO ALOMÉTRICO DE SPURR.

Abaixo a resolução das tarefas de 4 a 7

In [43]:
# FUNÇÕES DE PARA CALCULO DE MÉTRICAS

metricas <- function(observado, previsto, n){
     
    numerador = sum((observado - previsto)^2)
    denominador = sum((observado - mean(observado))^2)
    R2 = round(1 - (numerador/denominador), 4)
    syx = round(sqrt((numerador/(n-2))),4)
    syx_perc = round(syx/mean(observado)*100,2)
    
    cat("Métricas:","\nR² = ",R2,"\nSyx =",syx,"\nSyx(%) = ", syx_perc,"%")
}


#### 4.1 - RANDOM FOREST (rf)

In [44]:
# FORMULA GERAL
formula2 <- VOL~.

In [45]:
modelo_rf2 <- train(formula2, dados_treino2, method = "rf")

In [46]:
saveRDS(modelo_rf2, "modelo_rf2.rds")

In [47]:
modelo_rf2 <- readRDS("modelo_rf2.rds")

In [48]:
summary(modelo_rf2)

                Length Class      Mode     
call              4    -none-     call     
type              1    -none-     character
predicted        80    -none-     numeric  
mse             500    -none-     numeric  
rsq             500    -none-     numeric  
oob.times        80    -none-     numeric  
importance        4    -none-     numeric  
importanceSD      0    -none-     NULL     
localImportance   0    -none-     NULL     
proximity         0    -none-     NULL     
ntree             1    -none-     numeric  
mtry              1    -none-     numeric  
forest           11    -none-     list     
coefs             0    -none-     NULL     
y                80    -none-     numeric  
test              0    -none-     NULL     
inbag             0    -none-     NULL     
xNames            4    -none-     character
problemType       1    -none-     character
tuneValue         1    data.frame list     
obsLevels         1    -none-     logical  
param             0    -none-   

In [49]:
previsao_rf2 <- predict(modelo_rf2, dados_teste2)

In [50]:
observado_previsto1 <- cbind(dados_teste2, previsao_rf2)
observado_previsto1 <- observado_previsto1 %>%
    mutate(Erro = VOL - previsao_rf2)
head(observado_previsto1, 6)

X,DAP,HT,HP,VOL,previsao_rf2,Erro
6,29.9,27.10,1.90,0.9067022,0.8924613,0.01424091
13,32.6,27.70,2.50,1.0801767,1.1219640,-0.04178723
17,39.2,26.95,1.75,1.5938084,1.5523540,0.04145438
18,33.6,25.10,2.00,0.9667870,1.0943471,-0.12756017
27,28.5,25.75,2.65,0.8161506,0.9098041,-0.09365343
41,38.0,23.90,2.90,1.2233266,1.7071130,-0.48378633


In [51]:
metricas(observado_previsto1$VOL, observado_previsto1$previsao_rf2, nrow(dados_treino2))

Métricas: 
R² =  0.7438 
Syx = 0.0904 
Syx(%) =  6.74 %

#### 4.2 - SVM (svmRadial)

In [52]:
modelo_svm2 <- train(formula2, dados_treino2, method = "svmRadial")

In [53]:
saveRDS(modelo_svm2, "modelo_svm2.rds")

In [54]:
modelo_svm2 <- readRDS("modelo_svm2.rds")

In [55]:
summary(modelo_svm2)

Length  Class   Mode 
     1   ksvm     S4 

In [56]:
previsao_svm2 <- predict(modelo_svm2, dados_teste2)

In [57]:
observado_previsto2 <- cbind(dados_teste2, previsao_svm2)
observado_previsto2 <- observado_previsto2 %>%
    mutate(Erro = VOL - previsao_svm2)
head(observado_previsto2, 6)

X,DAP,HT,HP,VOL,previsao_svm2,Erro
6,29.9,27.10,1.90,0.9067022,0.9355955,-0.02889332
13,32.6,27.70,2.50,1.0801767,1.1142154,-0.03403867
17,39.2,26.95,1.75,1.5938084,1.5298574,0.06395096
18,33.6,25.10,2.00,0.9667870,1.1221910,-0.15540401
27,28.5,25.75,2.65,0.8161506,0.9538606,-0.13770993
41,38.0,23.90,2.90,1.2233266,1.4474814,-0.22415474


In [58]:
metricas(observado_previsto2$VOL, observado_previsto2$previsao_svm2, nrow(dados_treino2))

Métricas: 
R² =  0.7097 
Syx = 0.0963 
Syx(%) =  7.18 %

#### 4.3 - REDES NEURAIS (neuralnet)

In [ ]:
# Neural Network (method = 'neuralnet')
# For regression using package neuralnet with tuning parameters:
# Number of Hidden Units in Layer 1 (layer1, numeric)
# Number of Hidden Units in Layer 2 (layer2, numeric)
# Number of Hidden Units in Layer 3 (layer3, numeric)

In [59]:
modelo_rna2 <- train(formula2, 
                     dados_treino2, 
                     method = "neuralnet",
                     tuneGrid = expand.grid(layer1 = c(3), layer2 = c(2), layer3 = c(2)),
                     linear.output = TRUE)

In [60]:
saveRDS(modelo_rna2, "modelo_rna2.rds")

In [61]:
modelo_rna2 <- readRDS("modelo_rna2.rds")

In [62]:
print(modelo_rna2)

Neural Network 

80 samples
 4 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 80, 80, 80, 80, 80, 80, ... 
Resampling results:

  RMSE       Rsquared   MAE      
  0.3128508  0.4559649  0.2339832

Tuning parameter 'layer1' was held constant at a value of 3
Tuning
 parameter 'layer2' was held constant at a value of 2
Tuning parameter
 'layer3' was held constant at a value of 2


In [63]:
previsao_rna2 <- predict(modelo_rna2, dados_teste2)

In [64]:
observado_previsto3 <- cbind(dados_teste2, previsao_rna2)
observado_previsto3 <- observado_previsto3 %>%
    mutate(Erro = VOL - previsao_rna2)
head(observado_previsto3, 6)

X,DAP,HT,HP,VOL,previsao_rna2,Erro
6,29.9,27.10,1.90,0.9067022,0.8567127,0.04998948
13,32.6,27.70,2.50,1.0801767,1.1734810,-0.09330423
17,39.2,26.95,1.75,1.5938084,1.4930934,0.10071498
18,33.6,25.10,2.00,0.9667870,1.1651457,-0.19835869
27,28.5,25.75,2.65,0.8161506,0.9046061,-0.08845548
41,38.0,23.90,2.90,1.2233266,1.5614721,-0.33814547


In [65]:
metricas(observado_previsto3$VOL, observado_previsto3$previsao_rna2, nrow(dados_treino2))

Métricas: 
R² =  0.793 
Syx = 0.0813 
Syx(%) =  6.06 %

### 5 - O MODELO ALOMÉTRICO É DADO POR: Volume = b0 + b1 * dap² * Ht

              alom <- nls(VOL ~ b0 + b1*DAP*DAP*HT, dados, start = list(b0 = 0.5, b1 = 0.5))

In [ ]:
# nls{stats}
# Determine the nonlinear (weighted) least-squares estimates of the parameters of a nonlinear model.
# Determine as estimativas de mínimos quadrados não lineares (ponderadas) dos parâmetros de um modelo 
# não linear.

In [66]:
modelo_alometrico <- nls(VOL ~ b0 + b1*DAP*DAP*HT, dados_treino2, start = list(b0 = 0.5, b1 = 0.5))

In [67]:
saveRDS(modelo_alometrico, "modelo_alometrico.rds")

In [68]:
modelo_alometrico <- readRDS("modelo_alometrico.rds")

In [69]:
modelo_alometrico
summary(modelo_alometrico)

Nonlinear regression model
  model: VOL ~ b0 + b1 * DAP * DAP * HT
   data: dados_treino2
       b0        b1 
3.791e-02 3.844e-05 
 residual sum-of-squares: 1.518

Number of iterations to convergence: 2 
Achieved convergence tolerance: 1.327e-08


Formula: VOL ~ b0 + b1 * DAP * DAP * HT

Parameters:
    Estimate Std. Error t value Pr(>|t|)    
b0 3.791e-02  5.444e-02   0.696    0.488    
b1 3.844e-05  1.514e-06  25.393   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.1395 on 78 degrees of freedom

Number of iterations to convergence: 2 
Achieved convergence tolerance: 1.327e-08


In [70]:
previsao_alometrico <- predict(modelo_alometrico, dados_teste2)

In [71]:
observado_previsto4 <- cbind(dados_teste2, previsao_alometrico)
observado_previsto4 <- observado_previsto4 %>%
    mutate(Erro = VOL - previsao_alometrico)
head(observado_previsto4, 6)

X,DAP,HT,HP,VOL,previsao_alometrico,Erro
6,29.9,27.10,1.90,0.9067022,0.9692349,-0.06253270
13,32.6,27.70,2.50,1.0801767,1.1695404,-0.08936366
17,39.2,26.95,1.75,1.5938084,1.6298279,-0.03601948
18,33.6,25.10,2.00,0.9667870,1.1271959,-0.16040896
27,28.5,25.75,2.65,0.8161506,0.8419107,-0.02576005
41,38.0,23.90,2.90,1.2233266,1.3645552,-0.14122857


In [72]:
metricas(observado_previsto4$VOL, observado_previsto4$previsao_alometrico, nrow(dados_treino2))

Métricas: 
R² =  0.8414 
Syx = 0.0712 
Syx(%) =  5.31 %

### 6 - EFETUE AS PREDIÇÕES NOS DADOS DE TESTE.

### 7 - CRIE FUNÇÕES E CALCULE AS SEGUINTES MÉTRICAS ENTRE A PREDIÇÃO E OS DADOS OBSERVADOS.

Coeficiente de determinação (R²): Quanto mais perto de 1 melhor é o modelo;

Erro padrão da estimativa (Syx): esta métrica indica erro, portanto quanto mais perto de 0 melhor é o modelo.

Erro padrão da estimativa percentual (Syx%): esta métrica indica porcentagem de erro, portanto quanto mais perto de 0 melhor é o modelo.

**Os passos 6 e 7 foram feitos juntamente com os passos 4 e 5 no exercício acima do dataset de estimativa de volumes de árvores.**

### 8 - ESCOLHA DO MELHOR MODELO

O melhor modelo conforme R² foi o SVM com R² = 0.8998. 

#### TEXTO COM A JUSTIFICATIVA DOS RESULTADOS GERADOS

Neste trabalho foi possível perceber que os modelos Random Forest (rpart e rf) e SVM (svm) tiveram melhores resultados
que os demais. O modelo de rede neural (RNA) no primeiro exercício (classificação) apresentou um R² ruim e o do segundo
exercício (regressão), o neuralnet, obteve R² = 0.793. O R² do modelo alométrico teve um resultado de 0.8414 sendo um dos melhores
no segundo exercício. O melhor modelo do primeiro exercício foi o SVM e no quarto foi o RNA (regressão).